# COUNTLAND: python tutorial
### Samuel H. Church

The following packages are required to complete the tutorial

In [1]:
import sys
import logging

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pylab as plt
import matplotlib.cm as cm
plt.style.use('classic')
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

import seaborn as sns

import scanpy as sc

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white', color_map="viridis")

scanpy==1.8.2 anndata==0.7.8 umap==0.5.2 numpy==1.21.5 scipy==1.8.0 pandas==1.4.1 scikit-learn==1.0.2 statsmodels==0.13.2 pynndescent==0.5.6


## GET THE DATA

We have used the PBMC3k benchmark dataset.  
`countland` accepts an AnnData object, same as `scanpy` 

In [2]:
#!mkdir data
#!wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O data/pbmc3k_filtered_gene_bc_matrices.tar.gz
#!cd data; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz
#!mkdir write

adata = sc.read_10x_mtx(
    '../data/hg19/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                             # write a cache file for faster subsequent reading

adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`


... reading from cache file cache/..-data-hg19-matrix.h5ad


## IMPORT COUNTLAND
The code is located in `countland.py`

In [3]:
import countland as clnd

In [12]:
import importlib
importlib.reload(clnd)

<module 'countland' from '/Users/samuelchurch/Downloads/countland/countland-py/countland.py'>

## INITIALIZE COUNTLAND OBJECT

In [13]:
C = clnd.countland(adata)
print(C)

Initializing countland object

        countland object
        Count matrix has 2700 cells (rows) 
         and 32738 genes (columns)
        The fraction of entries that are nonzero is 0.0259
        


The count matrix is stored in `C.counts`

In [14]:
C.counts

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Note that most counts are zero for scRNA-seq data.

## EVALUATE CELL QUALITY

Checking the total number of counts and other measures per cell provides a measure of data quality. 

In addition, it can be helpful to see how many counts are derived from certain genes, such as mitochondrial genes (here those gene names start with "MT-").

In [15]:
C.ScoreCells(gene_string="^MT-")

In [16]:
C.cell_scores

,names,max_count_value,total_counts,counts_above0,counts_above1,counts_above10,unique_count_values,count_index,feature_counts
0,AAACATACAACCAC-1,76,2421,781,225,56,34,10,73
1,AAACATTGAGCTAC-1,142,4903,1352,399,92,52,10,186
2,AAACATTGATCAGC-1,171,3149,1131,355,50,34,9,28
3,AAACCGTGCTTCCG-1,114,2639,960,319,48,28,9,46
4,AAACCGTGTATGCG-1,40,981,522,113,13,18,4,12
...,...,...,...,...,...,...,...,...,...
2695,TTTCGAACTCTCAT-1,96,3461,1155,392,60,36,9,73
2696,TTTCTACTGAGGCA-1,77,3447,1227,397,70,35,9,32
2697,TTTCTACTTCCTCG-1,59,1684,622,194,30,26,10,37
2698,TTTGCATGAGAGGC-1,109,1024,454,131,9,18,5,21


## FILTER GENES AND CELLS

In [ ]:
C.SubsetGenes(np.sum(C.counts,axis=0) > 0) # only genes with counts detected


In [ ]:
C.SubsetCells(np.sum(C.counts,axis=1) > 0) # only cells with counts detected


You can return to the original count matrix at any time using
`C.RestoreCounts()`

## SUBSAMPLE GENES

Cells are not sequenced to standard sequencing depth. This is sometimes a problem for downstream comparisons.  

You can subsample all cells to a standard number of counts using `C.Subsample()`. The number of counts must not be larger than the minimum number per cell.

In [ ]:
np.random.seed(84095) # choose a random seed for reproducibility
C.Subsample(n_counts = 500)

## COMPARE GENES BY COUNTS

Using the subsampled count matrix, we can compare expression using several count-based assessments. These include:

- maximum number of counts observed across cells
- total counts per gene
- number of cells with counts above 0, 1, or 10
- number of unique count values
- count index = number of _n_ cells with _n_ counts

In [ ]:
C.ScoreGenes()
C.gene_scores.sort_values(by="unique_count_values",ascending=False)

In [ ]:
####  NEED TO UPDATE THE PLOT HERE 

unique_names = C.gene_scores.sort_values(by="total_counts",ascending=False).names.unique()
colors_dict = dict(zip(unique_names,np.append(sns.color_palette("tab10",10).as_hex(),np.repeat("black",len(C.gene_scores.names)-10)))) # colors to markers


## SELECT CERTAIN GENES (AKA PROJECTING ONTO SUBSPACE)

A common practice in single-cell analysis is to only use certain features (genes) for clustering. These can be selected by estimated parameter values, such as high variance. 

Using count-based methods, there may not be a need to filter genes. However, for speed of analysis here, we have focused on the top 500 genes ranked by the number of unique count values per gene.

In [ ]:
gene_names = C.gene_scores.sort_values(by="unique_count_values",ascending=False).head(500)['names']
gene_indices = np.where(np.isin(C.names_genes,gene_names))[0]
C.SubsetGenes(gene_indices)
C.SubsetCells(np.where(np.max(C.counts,axis=1)>0)[0]) # only cells with at least one count

## CLUSTER BY DOT PRODUCTS

The dot (or inner) product is a measure of alignment between vectors. In this case, it tells us how similar two cells are based on the proportions of counts, and scaled by the total counts per cell. A dot product of 0 indicates orthogonal cell vectors (no shared counts), larger values indicate aligned cell vectors.

In [ ]:
C.Dot()

Cell populations can be distinguished by clustering the matrix of pairwise dot products (contained in `C.dots`). This matrix is an unbounded affinity matrix. It is symmetric, and contains only integer values above 0. Spectral clustering is appropriate for this type of matrix.

The number of clusters depends on the system in question...

In [ ]:
## NEED A BETTER SYSTEM FOR DECIDING N CLUSTERS

C.Cluster(n_clusters=7)

## VISUALIZING WITH SPECTRAL EMBEDDING OF DOT PRODUCTS

We can use the dot product matrix to visualize cell similarity. This takes advantage of spectral embedding to plot cells in a space with reduced dimensionality.

In [ ]:
C.PlotEmbedding()

## IDENTIFY MARKER GENES

What makes a gene an ideal marker for a cluster may depend on downstream applications. For example, the ideal marker gene might be defined as the gene detected in all of the cells in a given cluster and none of the rest.

Under this definition, the top marker gene for each cluster can be identified by counting and comparing the number of non-zero cells.

In [ ]:
C.RankMarkerGenes(method='prop-zero')
gdf = C.marker_genes.loc[(C.marker_genes['cluster_label'] == 5)]
gdf

In [ ]:
gene_index = gdf.loc[(gdf['cluster_label'] == 5) & (gdf['rank'] == 1)]['gene index'].values
C.PlotMarker(gene_index)

Alternatively, the top marker genes for each cluster can be identified by ranking genes using the Wilcoxon rank-sum statistic.

In [ ]:
C.RankMarkerGenes(method='rank-sums')
gdf = C.marker_genes.loc[(C.marker_genes['cluster_label'] == 5)]
gdf

In [ ]:
gene_index = gdf.loc[(gdf['cluster_label'] == 5) & (gdf['rank'] == 1)]['gene index'].values 
C.PlotMarker(gene_index)

## GLM-PCA

An alternative approach for comparing cells using untransformed counts is generalized linear model based PCA, or GLM-PCA. This has been described for scRNA-seq data [here](https://doi.org/10.1186/s13059-019-1861-6) and implemented [here](https://github.com/willtownes/glmpca-py).

In [ ]:
#!pip install git+https://github.com/willtownes/glmpca-py.git@master
from glmpca import glmpca

res = glmpca.glmpca(C.counts.T,2,fam="poi") # embed in 2 dimensions and use a Poisson model

In [ ]:
plt.figure(figsize = (6,6))

g = sns.scatterplot(x = res['factors'][:,0], # embeddings are stored in res['factors']
                y = res['factors'][:,1],
                hue = C.cluster_labels,
                palette="tab10",
                s =10,
                linewidth=0)
g.legend(loc=(1.04,0))
g.xaxis.set_ticklabels([])
g.yaxis.set_ticklabels([])
g.set(title="GLM-PCA: dot product clusters")

In [ ]:
plt.figure(figsize = (6,6))

g = sns.scatterplot(x = res['factors'][:,0], 
            y = res['factors'][:,1], 
            color = "#CFCFCF",
            s = 5,
            linewidth=0)
sci = np.where(C.counts[:,gene_index]>0)[0]
sns.scatterplot(ax = g,
            x = res['factors'][sci,0], 
            y = res['factors'][sci,1], 
            hue = C.counts[sci,gene_index],
            palette = "viridis",
            s = 10,
            linewidth=0) 
g.legend(loc=(1.04,0))
g.xaxis.set_ticklabels([])
g.yaxis.set_ticklabels([])
g.set(title="GLM-PCA: dot product clusters")

## RECAPITULATING STANDARD APPROACHES

`countland` also includes functions for recapitulating the standard transformation steps for scRNA-seq data.
**Note that these are not recommended**

In [ ]:
C._Normalize()
C._Log()
C._RescaleVariance()
C._Center()